In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import csv
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.linearmodels as corplt
import numpy as np
import sklearn.linear_model
import time
from datetime import datetime
from numpy.linalg import inv
from sklearn.feature_extraction import DictVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

np.set_printoptions(precision=4, suppress=True)

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib64/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
with open('dailyRidesWeather.csv', 'rb') as f:
    reader = csv.reader(f)
    dat = list(reader)
    header = dat[0]
    rides = np.asarray(dat[1:])

print rides.shape

(149711, 32)


In [3]:
print rides[0]

['0' 'Yellow' '2016' '1' '15' '07114' '38.0' '1.1875' '32' '2016-01-15'
 'True' '9.50' 'False' '21.90' '51.10' '41.20' 'False' 'False' 'False'
 '0.00' '27.50' '-5.61' '10.61' '5.11' '1007.10' '1012.00' 'False' ''
 'False' '3.80' '8.00' '15.00']


In [4]:
counter = 0
feature_idx = {val:idx for idx, val in enumerate(header)}
# 0 : rnbr | 1 : ride_source | 2 : pickup_year | 3 : pickup_month | 4 : pickup_day | 5 : pickup_zipcode | 
# 6 : passenger_count | 7 : avg_passenger_count | 8 : ride_count | 9 : datestamp | 10 : weather_ok | 
# 11 : visibility | 12 : fog | 13 : min_temp | 14 : max_temp | 15 : mean_temp | 16 : rain | 17 : hail | 
# 18 : thunder | 19 : precipitation | 20 : dew_point | 21 : min_temp_c | 22 : max_temp_c | 23 : mean_temp_c | 
# 24 : station_pressure | 25 : sea_level_pressure | 26 : snow | 27 : snow_depth | 28 : tornado | 29 : mean_wind_speed | 
# 30 : max_wind_speed | 31 : max_wind_gus |

In [6]:
y_rides = rides[rides[:,1]=='Yellow']
y_labels = np.array([row[feature_idx['ride_count']] for row in y_rides], dtype=int)
g_rides = rides[rides[:,1]=='Green']
g_labels = np.array([row[feature_idx['ride_count']] for row in g_rides], dtype=int)

print y_rides.shape, g_rides.shape
print y_labels.shape, g_labels.shape

(87965, 32) (61746, 32)
(87965,) (61746,)


In [13]:
# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
y_shuffle = np.random.permutation(np.arange(y_rides.shape[0]))
g_shuffle = np.random.permutation(np.arange(g_rides.shape[0]))
y_rides, g_rides = y_rides[y_shuffle], g_rides[g_shuffle]
y_labels, g_labels = y_labels[y_shuffle], g_labels[g_shuffle]

# Split into train and test.
y_train_data, y_train_labels = y_rides[:80000], y_labels[:80000]
y_test_data, y_test_labels = y_rides[80000:], y_labels[80000:]

g_train_data, g_train_labels = g_rides[:55000], g_labels[:55000]
g_test_data, g_test_labels = g_rides[55000:], g_labels[55000:]

print y_train_data.shape, y_test_data.shape
print g_train_data.shape, g_test_data.shape

(80000, 32) (7965, 32)
(55000, 32) (6746, 32)


In [8]:
#Feature Extraction
def get_feature_dict(x):
    feature_dict = {}
    # Get pickup date
    pickup_date = datetime.strptime("%s-%s-%s" % (x[feature_idx["pickup_year"]],\
                                     x[feature_idx["pickup_month"]],\
                                     x[feature_idx["pickup_day"]]), '%Y-%m-%d')
    
    feature_dict["zipcode"] = x[feature_idx["pickup_zipcode"]]
    feature_dict["month"] = x[feature_idx["pickup_month"]].zfill(2)
    feature_dict["day"] = x[feature_idx["pickup_day"]].zfill(2)
    feature_dict["weekday"] = '%02d' % pickup_date.weekday()
    
    #mean temp
    if float(x[feature_idx["mean_temp"]]) < 55.:
        feature_dict["temp"] = "Cold"
    elif float(x[feature_idx["mean_temp"]]) > 75.:
        feature_dict["temp"] = "Hot"
    else:
        feature_dict["temp"] = "Normal"
        
    #mean wind speed
    feature_dict["wind_speed"] = "%0d" % (float(x[feature_idx["mean_wind_speed"]] or 12.))
        
    #mean wind speed
    feature_dict["precip"] = "%00d" % (float(x[feature_idx["precipitation"]] or 10.))
    
    #zipcode-weekday
    feature_dict["zipcode_weekday"] = "%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"])
    
    #zipcode-weekday-precip
    feature_dict["zipcode_weekday_precip"] = "%s_%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"], feature_dict["precip"])
    
    #zipcode-weekday-temp
    feature_dict["zipcode_weekday_temp"] = "%s_%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"], feature_dict["temp"])
    
    #zipcode-weekday-wind
    feature_dict["zipcode_weekday_wind"] = "%s_%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"], feature_dict["wind_speed"])
    
    #zipcode-weekday-wind-precip
    feature_dict["zipcode_weekday_temp_precip"] = "%s_%s_%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"], feature_dict["temp"], feature_dict["precip"])
    
    #zipcode-weekday-wind-precip
    feature_dict["zipcode_weekday_temp_precip_wind"] = "%s_%s_%s_%s_%s" % (feature_dict["zipcode"], feature_dict["weekday"], feature_dict["temp"], feature_dict["precip"], feature_dict["wind_speed"])
    
    return feature_dict

#print get_feature_dict(y_rides_2015[1])

In [10]:
y_vectorizer = DictVectorizer()
g_vectorizer = DictVectorizer()
def get_feature_vector(feat_dict, is_test, is_y):
    transformed = (y_vectorizer.transform(feat_dict) if is_test else y_vectorizer.fit_transform(feat_dict)) if is_y \
    else (g_vectorizer.transform(feat_dict) if is_test else g_vectorizer.fit_transform(feat_dict))
    return transformed

In [14]:
y_feature_dicts_train = [get_feature_dict(x) for x in y_train_data]
y_feature_dicts_test = [get_feature_dict(x) for x in y_test_data]

y_train_feature_vectors = get_feature_vector(y_feature_dicts_train, False, True)
y_test_feature_vectors = get_feature_vector(y_feature_dicts_test, True, True)

In [28]:
g_feature_dicts_train = [get_feature_dict(x) for x in g_train_data]
g_feature_dicts_test = [get_feature_dict(x) for x in g_test_data]

g_train_feature_vectors = get_feature_vector(g_feature_dicts_train, False, False)
g_test_feature_vectors = get_feature_vector(g_feature_dicts_test, True, False)

In [30]:
print len(g_vectorizer.feature_names_)

64430


In [ ]:
# using stocastic gradient descent.
y_sgd_regressor = linear_model.SGDRegressor(
    n_iter=5000, # Takes many iterations to converge.
    alpha=0.0, # Works better without regularization.
    learning_rate='invscaling',
    eta0=0.2, # Converges faster with higher-than-default initial learning rate.
    power_t=0.4,
    verbose=1
)
y_sgd_regressor.fit(y_train_feature_vectors, y_train_labels)

-- Epoch 1
Norm: 27442.00, NNZs: 120215, Bias: 5.854123, T: 80000, Avg. loss: 1397637.004082
Total training time: 0.02 seconds.
-- Epoch 2
Norm: 31933.97, NNZs: 120215, Bias: 5.250055, T: 160000, Avg. loss: 1063750.258951
Total training time: 0.04 seconds.
-- Epoch 3
Norm: 33995.94, NNZs: 120215, Bias: 4.330335, T: 240000, Avg. loss: 921027.857436
Total training time: 0.07 seconds.
-- Epoch 4
Norm: 35165.97, NNZs: 120215, Bias: 3.849654, T: 320000, Avg. loss: 841077.193588
Total training time: 0.09 seconds.
-- Epoch 5
Norm: 35906.87, NNZs: 120215, Bias: 3.399782, T: 400000, Avg. loss: 789531.453925
Total training time: 0.11 seconds.
-- Epoch 6
Norm: 36448.12, NNZs: 120215, Bias: 3.087101, T: 480000, Avg. loss: 753239.183845
Total training time: 0.13 seconds.
-- Epoch 7
Norm: 36842.35, NNZs: 120215, Bias: 2.884987, T: 560000, Avg. loss: 726125.832567
Total training time: 0.15 seconds.
-- Epoch 8
Norm: 37154.31, NNZs: 120215, Bias: 2.580710, T: 640000, Avg. loss: 704998.340523
Total trai

In [16]:
y_hat = y_sgd_regressor.predict(y_test_feature_vectors)

In [22]:
np.random.seed(0)
y_hat_trim = map(int,[x if x > 0 else np.random.randint(1,50) for x in y_hat])
print "Predicted"
print y_hat_trim[:100]

print "Actual"
print y_test_labels[:100]

print metrics.r2_score(y_test_labels, y_hat)
print metrics.r2_score(y_test_labels, y_hat_trim)

Predicted
[482, 4178, 45, 3097, 48, 2835, 33, 382, 954, 109, 237, 1, 421, 242, 1161, 4, 133, 603, 6, 233, 136, 6674, 89, 1061, 548, 107, 4, 40, 175, 408, 110, 171, 1141, 4558, 110, 190, 196, 6466, 10, 95, 161, 15, 4475, 298, 20, 160, 22, 113, 37, 172, 24, 7, 145, 25, 9351, 3602, 221, 25, 6251, 306, 13, 2, 79, 3896, 39, 129, 64, 40, 2403, 234, 7430, 136, 66, 24, 10458, 8668, 478, 47, 25, 93, 1324, 18, 114, 7796, 10784, 6463, 897, 194, 293, 84, 38, 26, 14, 8021, 529, 1823, 9, 122, 211, 574]
Actual
[ 1087  6610     5  3358     3  2901     1     2   900     4     5    19
   411     1  1382     2     7   791    10     6   148  6286    11  1268
   124     2     6     2    19     3     1     1  1479  4284     1     5
     4  8505     2    33     2     1  2900     6     1     2     1     1
     1    68     1     1     4     1  9649  2800     1     2  7785     1
     7    14     4  3741     1     1     5     1  2415     2  9006     7
     2     3    16     1     1    12    26    38     1     1 

In [ ]:
# Prediction model for green cab
# using stocastic gradient descent.
g_sgd_regressor = linear_model.SGDRegressor(
    n_iter=5000, # Takes many iterations to converge.
    alpha=0.0, # Works better without regularization.
    learning_rate='invscaling',
    eta0=0.2, # Converges faster with higher-than-default initial learning rate.
    power_t=0.4,
    verbose=1
)
g_sgd_regressor.fit(g_train_feature_vectors, g_train_labels)

-- Epoch 1
Norm: 5577.95, NNZs: 64430, Bias: 1.878704, T: 55000, Avg. loss: 43085.619469
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 6213.09, NNZs: 64430, Bias: 1.861180, T: 110000, Avg. loss: 29116.909457
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 6455.44, NNZs: 64430, Bias: 1.640345, T: 165000, Avg. loss: 23619.088886
Total training time: 0.04 seconds.
-- Epoch 4
Norm: 6593.23, NNZs: 64430, Bias: 1.697908, T: 220000, Avg. loss: 20628.964489
Total training time: 0.06 seconds.
-- Epoch 5
Norm: 6683.59, NNZs: 64430, Bias: 1.558227, T: 275000, Avg. loss: 18723.941999
Total training time: 0.07 seconds.
-- Epoch 6
Norm: 6748.06, NNZs: 64430, Bias: 1.509606, T: 330000, Avg. loss: 17390.287992
Total training time: 0.08 seconds.
-- Epoch 7
Norm: 6801.53, NNZs: 64430, Bias: 1.492908, T: 385000, Avg. loss: 16400.608148
Total training time: 0.10 seconds.
-- Epoch 8
Norm: 6844.52, NNZs: 64430, Bias: 1.476263, T: 440000, Avg. loss: 15632.793939
Total training time: 0.11 seconds.
-

In [32]:
# Predict
g_hat = g_sgd_regressor.predict(g_test_feature_vectors)

In [33]:
g_hat_trim = map(int,[x if x > 0 else np.random.randint(1,50) for x in g_hat])
print "Predicted"
print g_hat_trim[:100]

print "Actual"
print g_test_labels[:100]

print metrics.r2_score(g_test_labels, g_hat)
print metrics.r2_score(g_test_labels, g_hat_trim)

Predicted
[24, 7, 34, 25, 52, 38, 2151, 167, 11, 171, 205, 111, 114, 30, 48, 50, 171, 1487, 225, 296, 71, 413, 776, 8, 200, 96, 1174, 798, 1225, 154, 10, 388, 664, 598, 173, 5, 13, 88, 13, 448, 12, 438, 284, 15, 638, 120, 1807, 36, 29, 155, 85, 153, 15, 2, 25, 104, 4, 17, 44, 19, 791, 2, 186, 1059, 47, 386, 102, 5, 22, 78, 134, 79, 7, 9, 12, 21, 25, 130, 2, 57, 78, 687, 41, 22, 148, 28, 875, 83, 95, 27, 47, 743, 360, 19, 6, 10, 752, 1897, 1235, 527]
Actual
[  15    1    4    4   52    7 2057  104    5  156  221  135  114    6    1
    1  178    1  297  453   66  369  703    4  168   96 1106  820 1356  151
    1  391  523  652  160    2    7  127    2  415    2  430  292    1  727
  106 1335    9    7  158   87  113   17   26    4  150    3    2   52    4
  806    1  229 1113    1  310    1   17    2   76  110   57    4    1   20
    1    1  162    4   23  122  765    6    1  139    1  874   85    1    3
   61  855  334    1    1    1  787 2018 1152  522]
0.916291546907
0.916266450314


In [34]:
# Save model and vectorizer
from sklearn.externals import joblib
joblib.dump(y_sgd_regressor, 'nyc_yellow_taxi_predictor.pkl') 
joblib.dump(y_vectorizer, 'nyc_yellow_taxi_vectorizer.pkl')

joblib.dump(g_sgd_regressor, 'nyc_green_taxi_predictor.pkl') 
joblib.dump(g_vectorizer, 'nyc_green_taxi_vectorizer.pkl')

['nyc_green_taxi_vectorizer.pkl']